# Vitessce integration

For more comprehensive integration tests, see: https://github.com/laminlabs/lamin-spatial

In [ ]:
!lamin login testuser1
!lamin init --storage "s3://lamindb-ci/test-vitessce"

In [ ]:
import lamindb as ln

In [ ]:
pbmc68k = ln.core.datasets.anndata_pbmc68k_reduced()

In [ ]:
pbmc68k = pbmc68k[:100, :200].copy()

In [ ]:
zarr_filepath = "my_test.zarr"
pbmc68k.write_zarr(zarr_filepath)

In [ ]:
zarr_artifact = ln.Artifact(zarr_filepath, description="Test dataset")
zarr_artifact.save()

In [ ]:
from vitessce import (
    VitessceConfig,
    Component as cm,
    AnnDataWrapper,
)
vc = VitessceConfig(
    schema_version="1.0.15",
    name="Test dataset",
    description="Test description",
)
dataset = vc.add_dataset(name="test1").add_object(
    AnnDataWrapper(adata_url=zarr_artifact.path.to_url()),
)
obs_sets = vc.add_view(cm.OBS_SETS, dataset=dataset)
obs_sets_sizes = vc.add_view(cm.OBS_SET_SIZES, dataset=dataset)
scatterplot = vc.add_view(cm.SCATTERPLOT, dataset=dataset, mapping="UMAP")
heatmap = vc.add_view(cm.HEATMAP, dataset=dataset)
genes = vc.add_view(cm.FEATURE_LIST, dataset=dataset)
vc.layout(((scatterplot | obs_sets) / heatmap) | (obs_sets_sizes / genes))

# inspect the config
vc.to_dict()

In [ ]:
from lamindb.integrations import save_vitessce_config

In [ ]:
vc_artifact = save_vitessce_config(vc, description="View Habib17 in Vitessce")

In [ ]:
zarr_artifact.delete(permanent=True)
vc_artifact.delete(permanent=True)

In [ ]:
ln.setup.delete("test-vitessce", force=True)